# Projekte der OER-Strategie

Copyright 2025 Stephan Kulla ("Kulla")

Licensed under the Apache License, Version 2.0 (the "Apache License") and Creative Commons Attribution 4.0 International (the "CC-BY License"). You may choose either of these licenses to govern your use of this project.

You may obtain a copy of the Apache License at:
    http://www.apache.org/licenses/LICENSE-2.0

You may obtain a copy of the CC-BY License at:
    https://creativecommons.org/licenses/by/4.0/

Unless required by applicable law or agreed to in writing, software and content distributed under the Apache License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the Apache License for the specific language governing permissions and limitations under the License.

Under the CC-BY License, you are free to share and adapt the material provided you give appropriate credit, provide a link to the license, and indicate if changes were made. See the CC-BY License for full details.

Third-Party Components and Licenses:
This product also includes third-party components which are distributed under their respective licenses. Please refer to the NOTICE file for details.

## Utility functions and imports

In [1]:
%load_ext jupyter_ai_magics

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## Fetch all projects

In [24]:
import requests

url = "https://www.oer-strategie.de/wp-json/wp/v2/projects"

# Query parameters
params = {
    "_fields": "id,date,slug,link,title,meta,project_status,project_focus_areas,federate_states,addressees,legal_persons,funding_guidelines,content",
    "per_page": "100",
    "orderby": "id",
    "order": "asc",
    "page": "1"
}

# Headers
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:135.0) Gecko/20100101 Firefox/135.0",
    "Accept": "*/*",
    "Accept-Language": "de-DE,en-US;q=0.7,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.oer-strategie.de/foerdern/gefoerderte-projekte/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Cookie": "matomooptin=0",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Priority": "u=4",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "TE": "trailers"
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Print the response text
projects_json = response.json()
projects_json[0]

{'id': 2716,
 'date': '2024-12-11T10:48:33',
 'slug': 'co-woerk-projektsteckbrief',
 'link': 'https://www.oer-strategie.de/projects/co-woerk-projektsteckbrief/',
 'title': {'rendered': 'Community zum Wissenstransfer OER: Netzwerk von Bildungsakteur:innen'},
 'content': {'rendered': '\n<figure class="wp-block-table"><table><tbody><tr><td><strong>Akronym</strong></td><td>CO-WOERK</td></tr><tr><td><strong>Projekttitel</strong></td><td>Community zum Wissenstransfer OER: Netzwerk von Bildungsakteur:innen</td></tr><tr><td><strong>Förderrichtlinie</strong></td><td>OE_COM</td></tr><tr><td><strong>Laufzeit</strong></td><td>01.06.2024 – 31.05.2027</td></tr><tr><td><strong>Status</strong></td><td>laufend</td></tr><tr><td><strong>Ansprechpartner</strong></td><td>Europa-Universität Viadrina Frankfurt (Oder)<br>Dr. Christin Barbarino<br>Hochschule Neubrandenburg &#8211; University of Applied Sciences<br>Prof. Dr. Matthias Müller <br>E-Mail:<br>barbarino[ät]europa-uni.de<br>mueller[ät]hs-nb.de</td></

In [25]:
def get_project_info(raw_data):
    metadata = raw_data["meta"]
    
    return {
        "Titel": raw_data["title"]["rendered"],
        "Link": raw_data["link"],
        "Akronym": metadata["acronym"],
        "Startdatum": metadata["start_date"],
        "Enddatum": metadata["end_date"],
        "Projektstatus": raw_data["project_status"]
    }

projects = [get_project_info(raw_data) for raw_data in projects_json]
projects[0]

{'Titel': 'Community zum Wissenstransfer OER: Netzwerk von Bildungsakteur:innen',
 'Link': 'https://www.oer-strategie.de/projects/co-woerk-projektsteckbrief/',
 'Akronym': 'CO-WOERK',
 'Startdatum': '2024-06-01',
 'Enddatum': '2027-05-31',
 'Projektstatus': 'ongoing'}

## Add information from project site

In [27]:
import locale
from bs4 import BeautifulSoup

# Set the locale to German (Germany)
locale.setlocale(locale.LC_NUMERIC, 'de_DE.UTF-8')

def with_project_site_information(project):
    site_html_text = requests.get(project["Link"], headers=headers).text
    site_html = BeautifulSoup(site_html_text, "html.parser")

    data = {
        "Webseite: ": None,
        "Förderkennzeichen": None,
        "Fördersumme": None,
        "Förderrichtlinie": None
    }
    
    # Extracting specific fields
    for field in data.keys():
        element = site_html.find(string=field)
        if element:
            parent = element.find_parent().find_parent()
            if parent:
                next_sibling = parent.find_next_sibling()
                if next_sibling:
                    data[field] = next_sibling.get_text(strip=True)

    data["Webseite"] = data["Webseite: "]
    del data["Webseite: "]

    data["Fördersumme"] = locale.atof(data["Fördersumme"].rstrip(" €"))

    return project | data

projects = [with_project_site_information(project) for project in projects]
projects[0]

{'Titel': 'Community zum Wissenstransfer OER: Netzwerk von Bildungsakteur:innen',
 'Link': 'https://www.oer-strategie.de/projects/co-woerk-projektsteckbrief/',
 'Akronym': 'CO-WOERK',
 'Startdatum': '2024-06-01',
 'Enddatum': '2027-05-31',
 'Projektstatus': 'ongoing',
 'Förderkennzeichen': '01PO23013',
 'Fördersumme': 2725774.41,
 'Förderrichtlinie': 'OE_COM',
 'Webseite': 'https://www.co-woerk.de'}

## Output as CSV

In [28]:
import csv

# Specify the CSV file name
csv_file = '/tmp/oer_strategie_data.csv'

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    # Create a CSV DictWriter object
    writer = csv.DictWriter(file, fieldnames=projects[0].keys())

    # Write the header
    writer.writeheader()

    # Write the data
    for row in projects:
        writer.writerow(row)

print(f"Data has been successfully written to {csv_file}")

Data has been successfully written to /tmp/oer_strategie_data.csv
